In [ ]:
# === Notebook bootstrap: make repo root importable ===
import sys
from pathlib import Path

_cwd = Path.cwd().resolve()
for p in [_cwd, *_cwd.parents]:
    if (p / "src").exists():
        if str(p) not in sys.path:
            sys.path.insert(0, str(p))
        break

print("cwd:", _cwd)
print("sys.path[0]:", sys.path[0])


In [ ]:
# 01
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.io_utils import load_pickle, outputs_dir
from src.modeling import predict_annual_sales_52_from_df
from src.features import build_log_ad_from_spend
from src.simulate import (
    build_ws_52,
    build_df_sim_52,
    build_avg_df_as_is,
    build_step8_scenario_table_52,
    build_online_as_is_paths_52,
)

OUT_DIR = outputs_dir()

DF_PATH = Path(OUT_DIR) / "df_w_feat.pkl"
WS_PATH = Path(OUT_DIR) / "weekly_score.pkl"
MODEL_PATH = Path(OUT_DIR) / "final_model.pkl"
BEST_DECAY_PATH = Path(OUT_DIR) / "best_decay.pkl"

df_w = load_pickle(DF_PATH).copy()          # df_w_feat は calendar 済みの正本として扱う
ws = load_pickle(WS_PATH).copy()
final_model = load_pickle(MODEL_PATH)

print("df_w:", df_w.shape)
print("ws:", ws.shape)
print("exog:", final_model.model.exog_names)


In [ ]:
# 02

weekly_score = ws.copy()  # 正規化は build_ws_52 側でやる

# シナリオ設定
ONLINE_SHARE_LIST        = [0.7, 0.8]                  # Online:Broadcast = 7:3 / 8:2
ALWAYS_ON_SHARE_LIST     = [0.1, 0.2, 0.3, 0.4, 0.5]    # Online内Always-ON比率
BOOST_QUANTILE           = 0.6                          # Boost対象（上位40%）
BOT_MODE                 = "baseline"                  # Always-ON側はbaseline_hat比例
PULSE_WEEKS              = (16, 20)                     # week_of_yearで統一（=iso_week）
SIM_FY_USED              = 2022                         # 例：07/08で使っているFY（後のセルで必要ならここに寄せる）

best_decay = load_pickle(BEST_DECAY_PATH)
D_ONLINE = float(best_decay["d_online"])
D_BROADCAST = float(best_decay["d_broadcast"])
D_OOH = float(best_decay.get("d_ooh", 0.0))  # Step08では0固定でも良いが、整合のため読んでOK

print("decay used in Step08:", D_ONLINE, D_BROADCAST, D_OOH)


In [ ]:
# 03
# =========
# Prediction wrapper (Step08)
# =========

EXOG = list(final_model.model.exog_names)

def predict_annual_sales_52_local(
    df_sim_52: pd.DataFrame,
    online_path_52: np.ndarray,
    broadcast_path_52: np.ndarray,
    ooh_path_52: np.ndarray,
) -> float:
    tmp = df_sim_52.copy()

    tmp["log_ad_online"] = build_log_ad_from_spend(online_path_52, D_ONLINE)
    tmp["log_ad_broadcast"] = build_log_ad_from_spend(broadcast_path_52, D_BROADCAST)
    tmp["log_ad_ooh"] = build_log_ad_from_spend(ooh_path_52, D_OOH)

    return float(predict_annual_sales_52_from_df(final_model, tmp, EXOG))

In [ ]:
# 04
# =========
# ws_52: typical week table (1..52)
# =========
ws_52 = build_ws_52(weekly_score)

display(ws_52.head())
print("ws_52 range:", ws_52["week_of_year"].min(), "-", ws_52["week_of_year"].max(), "rows:", len(ws_52))


In [ ]:
# 05
# =========
# df_sim_52: 52-week template (ISO week-based, derived from df_w)
# =========

df_sim_52 = build_df_sim_52(df_w, sim_fy=SIM_FY_USED)

print("SIM_FY_USED =", SIM_FY_USED)
display(df_sim_52[["Week", "fy_year", "fy_week", "iso_week", "week_of_year"]].head())
print("df_sim_52 len:", len(df_sim_52))


In [ ]:
# 06
# =========
# Build avg_df (As-Is annual average) for Step08（FY=fy_yearベース）
# =========

MEDIA_COLS = ["broadcast_spend", "ooh_print_spend", "online_spend"]
MIN_WEEKS = 49

avg_df = build_avg_df_as_is(df_w, media_cols=MEDIA_COLS, min_weeks=MIN_WEEKS)

display(avg_df)


In [ ]:
# 07
# =========
# As-Is (baseline for index) -- 07/08一貫定義
# =========

n = len(df_sim_52)
assert n == 52, f"df_sim_52 must be 52 rows. got {n}"

(
    total_budget_as_is, avg_online, avg_br, avg_ooh,
    online_as_is_52, broadcast_as_is_52, ooh_as_is_52
) = build_online_as_is_paths_52(avg_df, n_weeks=n)

print("As-Is total budget:", f"{total_budget_as_is:,.0f}")
print("As-Is avg_online   :", f"{avg_online:,.0f}")
print("As-Is avg_br       :", f"{avg_br:,.0f}")
print("As-Is avg_ooh      :", f"{avg_ooh:,.0f}")

sales_as_is = predict_annual_sales_52_local(
    df_sim_52,
    online_as_is_52,
    broadcast_as_is_52,
    ooh_as_is_52,
)

as_is_roi_accounting = sales_as_is / total_budget_as_is

print("As-Is sales:", f"{sales_as_is:,.0f}")
print("As-Is accounting ROI (sales/total_budget):", f"{as_is_roi_accounting:.6f}")


In [ ]:
# 08
# =========
# Scenario table (Step08) - 52週ベースで統一 + ROI score追加
# =========

step8_df = build_step8_scenario_table_52(
    df_sim_52=df_sim_52,
    ws_52=ws_52,
    avg_df=avg_df,
    online_share_list=ONLINE_SHARE_LIST,
    always_on_share_list=ALWAYS_ON_SHARE_LIST,  # ※ Cell02で統一した名前
    boost_quantile=BOOST_QUANTILE,              # ※ Cell02で統一した名前
    pulse_weeks=PULSE_WEEKS,
    bot_mode=BOT_MODE,
    predict_annual_sales_52_fn=predict_annual_sales_52_local,
)

display(step8_df)

# 保存
step8_path = OUT_DIR / "scenario_table.csv"
step8_df.to_csv(step8_path, index=False)
print("Saved:", step8_path)
